# 小説の文章スタイルを分析する

ライブラリのインストール

In [ ]:
!apt-get install -y mecab libmecab-dev mecab-ipadic-utf8
!pip install unidic-lite --quiet
!pip install mecab-python3 --quiet
!pip install Levenshtein --quiet
#!pip install nltk --quiet


ドライブのマウント


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


テキストファイルの文章から'動詞', '形容詞', '副詞', '連体詞', '接続詞'の単語を抜き出す。

元のテキストファイル：input_folder  
作成したテキストファイル：output_folder


In [ ]:
import MeCab
import os
import re
import chardet

# ドライブ内のファイルが格納されたフォルダのパス
input_folder = '/content/drive/MyDrive/Mago_Serise/mago121(sentence_style)/novel_style/C_00/'
output_folder = '/content/drive/MyDrive/Mago_Serise/mago121(sentence_style)/novel_style/C_01'

# MeCabの初期化
tagger = MeCab.Tagger()

# 入力フォルダ内のすべてのテキストファイルを処理
for filename in os.listdir(input_folder):
    if filename.endswith(".txt"):
        # テキストファイルのパス
        input_file_path = os.path.join(input_folder, filename)

        # テキストファイルから文章を読み込む（エンコーディングを自動検出）
        with open(input_file_path, 'rb') as file:
            rawdata = file.read()
        encoding = chardet.detect(rawdata)['encoding']
        text = rawdata.decode(encoding)

        text = re.sub("《[^》]+》", "", text)  # ルビの削除
        text = re.sub("［[^］]+］", "", text)  # 読みの注意の削除
        text = re.sub("【([^】]*)】", "", text)  # 読みの注意の削除
        text = re.sub("[｜ 　]", "", text)  # | と全角半角スペースの削除

        # 形態素解析を行い、品詞を抽出して新しい文章を生成
        new_sentence = []
        node = tagger.parseToNode(text)
        while node:
            features = node.feature.split(',')
            if features[0] in ['動詞' '形容詞', '副詞', '連体詞', '接続詞']:
                new_sentence.append(node.surface)
            node = node.next

        # 単語を一文字分のスペースで区切って結合
        new_sentence_str = '　'.join(new_sentence)

        # 出力ファイルのパス
        output_file_path = os.path.join(output_folder, filename)

        # 新しい文章をテキストファイルに書き出す
        with open(output_file_path, 'w', encoding='utf-8') as file:
            file.write(new_sentence_str)


単語の出現頻度表を作成


In [ ]:
import os
import MeCab
import pandas as pd
from collections import Counter

# ドライブ内のファイルが格納されたフォルダのパス
folder_path = '/content/drive/MyDrive/Mago_Serise/mago121(sentence_style)/novel_style/C_01'


# MeCabの初期化
m = MeCab.Tagger("-Owakati")

# 形態素と頻度を格納する辞書
word_freq_dict = {}

# ファイルごとに形態素解析と単語の頻度を計算
file_list = os.listdir(folder_path)

# すべての単語を含むリストを作成
unique_words = []

for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)

    with open(file_path, 'rb') as file:
        text = file.read().decode('utf-8', errors='replace')  # 'utf-8'に変更

    # 形態素解析して単語リストを取得
    parsed_text = m.parse(text)
    words = parsed_text.split()

    # 形態素の頻度を計算
    word_freq = Counter(words)

    # すべての単語をリストに追加
    unique_words.extend(word_freq.keys())

    # 辞書に追加
    word_freq_dict[file_name] = word_freq

# 一意の単語リストを作成
unique_words = list(set(unique_words))

# 各ファイルの単語頻度を持つデータフレームを作成
df = pd.DataFrame(word_freq_dict).fillna(0)

# 一意の単語リストを持つデータフレームを作成
word_df = pd.DataFrame(index=unique_words)

# 一意の単語リストを持つデータフレームとファイルごとの単語頻度のデータフレームを結合
result_df = word_df.join(df)

# NaNを0に置換
result_df = result_df.fillna(0)

# 結果を表示
print(result_df)


In [ ]:
# 結果をCSVファイルとして保存
result_path = '/content/drive/MyDrive/Mago_Serise/mago121(sentence_style)/novel_style/result' #パスを指定
df.to_csv(result_path + '/adjectives_adverbs_freq.csv', encoding='utf-8')


多次元尺度法で表示


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import MDS
from sklearn.metrics import euclidean_distances
from Levenshtein import distance as levenshtein_distance

# CSVファイルからデータを読み込む
data = pd.read_csv(result_path + '/adjectives_adverbs_freq.csv')

# 1列目を除いた名前のリストを取得
names = data.columns.tolist()[1:]

# 名前の類似度に基づいて距離行列を計算
distances = np.zeros((len(names), len(names)))
for i in range(len(names)):
    for j in range(len(names)):
        distances[i, j] = levenshtein_distance(names[i], names[j])

# 多次元尺度法を適用して2次元の配置を取得
mds = MDS(n_components=2, dissimilarity='precomputed', random_state=42)
pos = mds.fit_transform(distances)

# 結果をプロット
plt.figure(figsize=(10, 8))
plt.scatter(pos[:, 0], pos[:, 1], color='y', s=100)

# 各点に名前を表示
for i, txt in enumerate(names):
    plt.annotate(txt, (pos[i, 0], pos[i, 1]))

plt.title('Multidimensional Scaling Plot of Novels')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.grid(True)
plt.show()
